### How to set up TPU 
#turn the accelerator to TPU 
### detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

### instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

### instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    model = tf.keras.Sequential( … ) # define your model normally
    model.compile( … )

### train model normally
model.fit(training_dataset, epochs=EPOCHS, steps_per_epoch=…)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib notebook
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf

I. Analyse des données 

In [ ]:
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
# Detect hardware, return appropriate distribution strategy
# Setting up TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
encode_sentence("I love machine learning"), encode_sentence("I love mach learning")

### Constatations : 
- Aucune données ne manquent pour les 12120 entrées
- Les données d'entrainement contiennent 15 langues différentes, d'abréviations 2 à 2 distinctes.
- La distribution des (entraînemet,neutralité, contradiction) est presque équitable entre les données.
- L'anglais est la langue la plus domainante (plus de 56 %).
- les données d'hypothèses sont 2 à 2 distincts (à part "I'm not sure.' de fréquence 2 ).


In [ ]:
train = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")
test = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/test.csv")

In [ ]:
train.columns

In [ ]:
len(train["lang_abv"].unique()), len(train["language"].unique())

In [ ]:
set(test["lang_abv"])== set(train["lang_abv"]), set(test["language"]) == set(train["language"])

In [ ]:
train.describe(include = "O"), train.describe()

distribution des catégories 

In [ ]:
#Label = train["label"].map({0: "entraînement" , 1 : "neutralité" , 2 : "contradiction"})
#Label = train["label"].map({"entraînement":0 ,  "neutralité":  1, "contradiction":2})

#train["label"] = Label

In [ ]:
train["label"].head()

In [ ]:
train["lang_abv"].unique()

In [ ]:
train["lang_abv"].groupby(train["lang_abv"]).count()

In [ ]:
languages = list(train["language"].unique())

In [ ]:
plt.clf()
plt.hist(train[train["language"]=="English"]["label"], color ="g")
plt.title("English")
plt.show()    

In [ ]:
colors = ["b","g","r","c","m"]
for i in range(len(languages)):
    plt.subplot(3,5,i+1)
    plt.hist(train[train["language"]==languages[i]]["label"], color = colors[i%5])
    plt.title(languages[i])
    plt.show()    


#### Distribution des phrases selon la langue 

In [ ]:
plt.clf()
languages = train["language"].unique()
lang_freq = {i : len(train[train["language"] == i]) for i in languages}
plt.figure(figsize = (8.5,8.5))
plt.pie(lang_freq.values(), labels = languages,autopct = '%1.0f%%')
plt.title("Distribution des phrases selon la langue")
plt.show()

In [ ]:
# Pour les données de test 
plt.clf()
languages_test = test["language"].unique()
lang_freq_test = {i : len(test[test["language"] == i]) for i in languages_test}
plt.figure(figsize = (8.5,8.5))
plt.pie(lang_freq_test.values(), labels = languages_test,autopct = '%1.0f%%')
plt.title("Distribution des phrases selon la langue (données de test)")
plt.show()


In [ ]:
test["language"].describe()

 ### Distribution des catégories selon la langue 

In [ ]:
len(train["lang_abv"].unique())

In [ ]:
# Changement des langues en valeurs numériques : 
Lang_abv = train["lang_abv"].map({train["lang_abv"].unique()[i] : i for i in range(len(train["lang_abv"].unique()))})
Languages = train["language"].map({train["language"].unique()[i] : i for i in range(len(train["lang_abv"].unique()))})

In [ ]:
train["lang_abv"] = Lang_abv
train["language"] = Languages

In [ ]:
#sns.pairplot(train[train["language"]=="English"], hue = "lang_abv")
plt.clf()
corrMatrix = train.corr()
sns.heatmap(corrMatrix, annot=True)

## A. Traitement des données 

In [ ]:
train.columns

In [ ]:
help(plt.hist)

In [ ]:
# taille des phrases 
prem, hyp = train["premise"], train["hypothesis"]
dict_label  ={0 :"entrainement", 1 : "neutre", 2 :"contradictoire"}
for i in range(3): 
    ls_prem = train[train["label"]==i]["premise"].str.split().apply(lambda x: len(x))
    ls_hyp = train[train["label"]==i]["hypothesis"].str.split().apply(lambda x: len(x))
    plt.subplot(3,2,1 +2*i)
    plt.hist(ls_prem, bins = 20)
    plt.xlabel("premise")
    plt.title(dict_label[i])
    plt.subplot(3,2, 2 + 2*i)
    plt.hist(ls_hyp, bins = 20)
    plt.title(dict_label[i])
    plt.xlabel("hypothesis")


On peut constater que les phrases sont plus longues pour les données premise par rapport a hypothesis et la distribution est presque la même pour les 3 classes. on peut s'en assurer en tracant la correlation 

In [ ]:
plt.clf()
copy_train = train
copy_train["length_sen_pre"] = prem.str.split().apply(lambda x : len(x))
copy_train["length_sen_hyp"] = hyp.str.split().apply(lambda x : len(x))
max_length = max(max(copy_train["length_sen_hyp"]),max(copy_train["length_sen_pre"]))
corrMatrix = copy_train.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
# taille maximum des phrases
max_length

#### Les stopwords 
On peut éliminer tous les stopwords des phrases, mais puique notre classification ne se base pas que sur la frésuence des mots, mais la relation entre les phrases, on doit faire attention aux stopwords de négativité qui change complétement le sens.

In [ ]:
# Analyse de l'impact des stopwords
import gensim.parsing.preprocessing as g 
stopWords = g.STOPWORDS
#on elimine les stopwords negatives
neg = ["no" , "not" , "has" ,"hasn't", 'did',
           'didn',
           'do',
           'does',
           'doesn',
           'doing',
           'don', 'could',
           'couldnt','can',
           'cannot',
           'cant', 'never', 'none']
stop_without_neg  =[word for word in stopWords if word not in neg]


In [ ]:
help(g.STOPWORDS)

In [ ]:
copy_train["if_neg_exist_prem"] = prem.str.split().apply(lambda x : len([word for word in neg if word in x]))
copy_train["if_neg_exist_hyp"] = hyp.str.split().apply(lambda x : len([word for word in neg if word in x]))

In [ ]:
plt.clf()
for i in range(3): 
    plt.subplot(3,2,1 +2*i)
    plt.hist(copy_train[copy_train["label"] ==i]["if_neg_exist_prem"]) # anglais seulement
    plt.xlabel("premise")
    plt.title(dict_label[i])
    plt.subplot(3,2, 2 + 2*i)
    plt.hist(copy_train[copy_train["label"] ==i]["if_neg_exist_hyp"])
    plt.title(dict_label[i])
    plt.xlabel("hypothesis")
plt.show()

In [ ]:
plt.clf()
corrMatrix = copy_train.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
#before
copy_train["premise"].iloc[0]

# eliminer les stopwords
#pour l'anglais 
copy_train["premise"] = prem.str.split().apply(lambda x : (" ").join([word for word in x if word not in stop_without_neg]))
test["premise"] = test["premise"].str.split().apply(lambda x : (" ").join([word for word in x if word not in stop_without_neg]))
# after
copy_train["premise"].iloc[0]
copy_train["hypothesis"] = hyp.str.split().apply(lambda x : (" ").join([word for word in x if word not in stop_without_neg]))
test["hypothesis"] = test["hypothesis"].str.split().apply(lambda x : (" ").join([word for word in x if word not in stop_without_neg]))
copy_train["length_sen_pre"] = prem.str.split().apply(lambda x : len(x))
copy_train["length_sen_hyp"] = hyp.str.split().apply(lambda x : len(x))
max_length = max(max(copy_train["length_sen_hyp"]),max(copy_train["length_sen_pre"]))

In [ ]:
#les autres 
from nltk.corpus import stopwords
stop_english = stopwords.words('english')
stop_french = stopwords.words('french')
stop_arabic = stopwords.words('arabic')
#stop_chinese = stopwords.words('chinese')
stop = []
stop.extend(stop_english)
stop.extend(stop_french)
stop.extend(stop_arabic)
copy_train["premise"] = train_set["premise"].apply(lambda x : [mot for mot in x.split(" ") if mot not in stop])
copy_train["hypothesis"] = train_set["hypothesis"].apply(lambda x : [mot for mot in x.split(" ") if mot not in stop])
copy_test["premise"] = test_set["premise"].apply(lambda x : [mot for mot in x.split(" ") if mot not in stop])
copy_test["hypothesis"] = test_set["hypothesis"].apply(lambda x : [mot for mot in x.split(" ") if mot not in stop])

In [ ]:
# eliminer ponctuations 
# before 
copy_train["premise"].iloc[0]

In [ ]:
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
copy_train["premise"].iloc[0]
def eliminate_punct(string):
    for elem in string : 
        if elem in punc : 
            string = string.replace(elem, "")
    return string
copy_train["premise"] = copy_train["premise"].apply(lambda x : eliminate_punct(str(x)))
copy_train["hypothesis"] = copy_train["hypothesis"].apply(lambda x : eliminate_punct(str(x)))
test["premise"] = test["premise"].apply(lambda x : eliminate_punct(str(x)))
test["hypothesis"] = test["hypothesis"].apply(lambda x : eliminate_punct(str(x)))

In [ ]:
# after
copy_train["premise"].iloc[0]

In [ ]:
# before
copy_train["premise"].iloc[1]

In [ ]:
#eliminer majiscule
copy_train["premise"] = copy_train["premise"].apply(lambda x : x.lower())
copy_train["hypothesis"] = copy_train["hypothesis"].apply(lambda x : x.lower())
test["premise"] = test["premise"].apply(lambda x : x.lower())
test["hypothesis"] = test["hypothesis"].apply(lambda x : x.lower())

In [ ]:
# after
copy_train["premise"].iloc[1]

#### Mots communs 
Intuitivement les phrases qui s'entraine ou sont contradictoire doivent avoir le nombre de mots communs important par rapport à celles neutres.

In [ ]:
# mots communs 
def commun_words(st1, st2):
    listst1 = st1.split(" ")
    listst2 = st2.split(" ")
    nb = 0
    for w in listst2: 
        if w in listst1: 
            nb += 1 
            listst1.remove(w)
    return nb 
            
communs  = []
for i in range(len(copy_train["premise"])):
    communs.append(commun_words(copy_train["premise"].iloc[i],copy_train["hypothesis"].iloc[i]))
copy_train["length_communs"] = communs

In [ ]:
plt.clf()
for i in range(3): 
    plt.subplot(3,2,1 +2*i)
    plt.hist(copy_train[copy_train["label"]==i]["length_communs"], bins = 30) # anglais seulement
    plt.title(dict_label[i])
plt.show()

In [ ]:
plt.clf()
corrMatrix = copy_train[["length_communs","label"]].corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

### Word Embeddings 

### Modèles
--> simple RNN
--> LSTM 
--> GRU 
--> BERT 

### BERT  

In [ ]:
from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# BERT pour contextualiser les embeddings 
#model_name = 'bert-large-cased'
model_name = "bert-base-multilingual-cased"
#tokenizer = BertTokenizer.from_pretrained(model_name) # KM: this is the tokenizer we will use on our text data to tokenize it

In [ ]:
# on utilise le tokenasiation de bert qui prend en consideration le contexte des mots dans la phrase
def encode_sentence(s):
    tokens = list(tokenizer.tokenize(s)) # KM: split the sentence into tokens that are either words or sub-words
    tokens.append('[SEP]') # KM: a token called [SEP] (=separator) is added to mark end of each sentence
    #return tokens
    return tokenizer.convert_tokens_to_ids(tokens) # KM: instead of returning the list of tokens, a list of each token ID is returned

In [ ]:
max_len= 210# valeur très grande
def bert_encode(hypotheses, premises, tokenizer):
    
    num_examples = len(hypotheses)
    #tokenazitation de toutes les phrases du hypothese et premise
    sentence1 = tf.ragged.constant([   # KM: constructs a constant ragged tensor. every entry has a different length (facilite de faire des operations sur less listes imbriquées)
      encode_sentence(s)
      for s in np.array(hypotheses)])
    sentence2 = tf.ragged.constant([
      encode_sentence(s)
       for s in np.array(premises)])
    # cls : indique le debut de phrase (apprait lg_phrase fois)
    cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*sentence1.shape[0] # KM: list of IDs for the token '[CLS]' to denote each beginning
    input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1) # KM: put everything together. every row still has a different length.
    # les mots inutils ajouté juste pour avoir des phrases de meme longeur sont codé en 0 , les autres en 1 
    input_mask = tf.ones_like(input_word_ids).to_tensor(shape =(input_word_ids.shape[0], max_len)) # KM: first, a tensor with just ones in it is constructed in the same size as input_word_ids. Then, by applying to_tensor the ends of each row are padded with zeros to give every row the same length

    type_cls = tf.zeros_like(cls) # KM: creates a tensor same shape as cls with only zeros in it
    type_s1 = tf.zeros_like(sentence1)
    type_s2 = tf.ones_like(sentence2) # KM: creates a tensor same shape as sentence2 with only ones in it to mark the 2nd sentence
    input_type_ids = tf.concat(
      [type_cls, type_s1, type_s2], axis=-1).to_tensor(shape =(input_word_ids.shape[0], max_len)) # KM: concatenates everything and again adds padding 

    inputs = {
      'input_word_ids': input_word_ids.to_tensor(shape =(input_word_ids.shape[0], max_len)), # KM: input_word_ids hasn't been padded yet - do it here now
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

    return inputs

max_len = 64
def encode_sentence(s, tokenizer):
    """
    Turn a sequence of words into and array of numbers using a selected tokenizer.
    Args:
        s (list of str) - Input string.
        tokenizer - XLM-R tokenizer.
    Returns:
        (list of int) - Tokenized string.

    """
    tokens = list(tokenizer.tokenize(s))
    tokens.append(tokenizer.sep_token)
    return tokenizer.convert_tokens_to_ids(tokens)

def tokenize(data, tokenizer, max_len):
    """
    Encode hypotheses and premises into arrays of numbers using a selected tokenizer. 
    Args:
        data - An array consisting of [hypothesis (str), premise (str)] pairs.
        tokenizer - Tokenizer handle.
        max_len - Max sequence length.
    Returns: (dictionary of tensors)
        input_word_ids - Indices of input sequence tokens in the vocabulary, truncated to max_len.
        input_mask - Real input indices mapped to ones. Padding indices mapped to zeroes.
        input_type_ids - Segment token indices to indicate first and second portions of the inputs.
    """

    PAD_ID = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
 
    # Append a separator to each sentence, tokenize, and concatenate.
    tokens1 = tf.ragged.constant([encode_sentence(s[0], tokenizer) for s in data], dtype=tf.int32) # ENCODED_SEQUENCE_A [SEP]
    tokens2 = tf.ragged.constant([encode_sentence(s[1], tokenizer) for s in data], dtype=tf.int32) # ENCODED_SEQUENCE_B [SEP]
    cls_label = [tokenizer.convert_tokens_to_ids([tokenizer.cls_token])]*tokens1.shape[0] # [CLS] ENCODED_SEQUENCE_A [SEP]
    tokens = tf.concat([cls_label, tokens1, tokens2], axis=-1) # [CLS] ENCODED_SEQUENCE_A [SEP] ENCODED_SEQUENCE_B [SEP]

    # Truncate to max_len.
    tokens = tokens[:, :max_len]

    # Pad with zeroes if len < max_len.
    tokens = tokens.to_tensor(default_value=PAD_ID)
    pad = max_len - tf.shape(tokens)[1]
    tokens = tf.pad(tokens, [[0, 0], [0, pad]], constant_values=PAD_ID)
    input_word_ids = tf.reshape(tokens, [-1, max_len])

    # The input mask allows the model to cleanly differentiate between the content and the padding. 
    input_mask = tf.cast(input_word_ids != PAD_ID, tf.int32)
    input_mask = tf.reshape(input_mask, [-1, max_len])

    # Map tokens1 indices to zeroes and tokens2 indices to ones.
    input_type_ids = tf.concat([tf.zeros_like(cls_label), tf.zeros_like(tokens1), tf.ones_like(tokens2)], axis=-1).to_tensor()


    inputs = {
      'input_word_ids': input_word_ids,
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

    return inputs

x_train, y_train = copy_train[['premise', 'hypothesis']].values.tolist(), copy_train['label']
train_input = tokenize(x_train,tokenizer, max_len)

len(copy_train.columns)

In [ ]:
# on construit le modele 
def build_model():
    bert_encoder = TFBertModel.from_pretrained(model_name) # KM: constructs a BERT model pre-trained on the above described language model 'bert-base-multilingual-cased'
    
    # KM: now we adjust the model so that it can accept our input by telling the model what the input looks like:
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids") # KM: tf.keras.Input constructs a symbolic tensor object whith certain attributes: "shape" tells it that the expected input will be in batches of max_len-dimensional vectors; "dtype" tells it that the data type will be int32; "name" will be the name string for the input layer
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask") # KM: repeat the same for the other two input variables
    input_type_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")
    
    # KM: now follows, what we want to happen with our input:
    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0] # KM: first, our input goes into the BERT model bert_encoder. It will return a tuple and the contextualized embeddings that we need are stored in the first element of that tuple
    x = tf.keras.layers.Dense(512, activation ="relu")(embedding[:,0,:])
    output = tf.keras.layers.Dense(3, activation='softmax')(x) # KM: as described below, we only need the output corresponding to the first token [CLS], which is a 2D-tensor with size (#sentence pairs, 768) and is accessd with embedding[:,0,:]. This will be input for our classifier, which is a regular densely-connected neural network constructed through tf.keras.layers.Dense. The inputs mean: "3" is the dimensionality of the output space, which means that the output has shape (#sentence pairs,3). More practically speaking, for each sentence pair that we input, the output will have 3 probability values for each of the 3 possible labels (entailment, neutral, contradiction). They will be in range(0,1) and add up to 1; "activation" denotes the activation function, in this case 'softmax', which connects a real vector to a vector of categorical possibilities.
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output) # KM: based on the code in the lines above, a model is now constructed and passed into the variable model
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy']) # KM: we tell the model how we want it to train and evaluate: "tf.keras.optimizers.Adam": use an optimizer that implements the Adam algorithm. "lr" denotes the learning rate; "loss" denotes the loss function to use; "metrics" specifies which kind of metrics to use for training and testing
    
    return model #KM

### Roberta XLM 

from transformers import BertTokenizer, AutoTokenizer, TFBertModel, TFXLMRobertaModel
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Embedding, GlobalAveragePooling1D

encoder_handle = 'jplu/tf-xlm-roberta-large'
tokenizer = AutoTokenizer.from_pretrained(encoder_handle)
random_seed = 11887
learning_rate = 1e-5 # Controls how large a step is taken when updating model weights during training.

def build_model2():
    tf.keras.backend.clear_session()
    tf.random.set_seed(random_seed)
    
    with strategy.scope():
        input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
        input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
        # RoBERTa doesn’t use token_type_ids.

        #  Create an instance of a model defined in encoder_handle
        roberta = TFXLMRobertaModel.from_pretrained(encoder_handle)
        roberta = roberta([input_word_ids, input_mask])[0]
        out = GlobalAveragePooling1D()(roberta)
        out = Dense(3, activation='softmax')(out)

        model = tf.keras.Model(inputs=[input_word_ids, input_mask], outputs = out)
        model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
model = build_model2() # KM: our model is being built
model.summary()       # KM: let's look at some of its properties

In [ ]:
model.fit(train_input, train.label.values, epochs = 5, verbose = 1, batch_size = 16 * strategy.num_replicas_in_sync, validation_split = 0.2)

x_test = test[['premise','hypothesis']].values.tolist()
test_input = tokenize(x_test, tokenizer, max_len)

In [ ]:
predictions = [np.argmax(i) for i in model.predict(test_input)]

submission = test.id.copy().to_frame()
submission['prediction'] = predictions
submission.to_csv("submission.csv", index = False)

#### simple RNN

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
import tensorflow as tf

# Detect hardware, return appropriate distribution strategy
# Setting up TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

tokenization

In [ ]:
token = text.Tokenizer(num_words=None)
max_len = 250 # taille maximale de phrases
separator = ' [SEP] ' #separer les deux phrases 
copy_train["concatenated"] = copy_train["premise"] + separator + copy_train["hypothesis"]
test["concatenated"] = test["premise"] + separator + test["hypothesis"]
token.fit_on_texts(list(copy_train["concatenated"]) + list(test["concatenated"]))
test_seq = token.texts_to_sequences(list(test["concatenated"]))
#zero pad the sequences : toutes les phrases deviennet de meme longueur(fit avec 0 )
test_pad = sequence.pad_sequences(test_seq, maxlen=max_len)
#A revoir !! 
train_seq = token.texts_to_sequences(list(copy_train["concatenated"]))
#zero pad the sequences : toutes les phrases deviennet de meme longueur(fit avec 0 )
train_pad = sequence.pad_sequences(train_seq, maxlen=max_len)
word_index = token.word_index 

In [ ]:
len(word_index)

model = Sequential()
model.add(Embedding(len(word_index) + 1, 300, input_length=max_len))
model.add(SimpleRNN(200))
model.add(Dense(3, activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
model.summary()

from keras.utils import to_categorical 
# on fait une premiere classification binaire (neutre ou en relation) 
#copy_train["label_bin"] = copy_train["label"].apply(lambda x : x%2) # 2 ==> 0
label =  to_categorical(copy_train["label"],3)
model.fit(train_pad, label, epochs=5, batch_size=64*strategy.num_replicas_in_sync) #Multiplying by Strategy to run on TPU's

predictions = [np.argmax(i) for i in model.predict(test_pad)]

submission = test.id.copy().to_frame()
submission['prediction'] = predictions 
submission.to_csv("submission.csv", index = False)

### LSTM 

simple RNN donne des résultats très médiocres vue que les phrases sont très longues , on essaiera d'améliorer cela en utilisatn un modèle à base LSTM 

#### Embeddings 

In [ ]:
from keras.utils import to_categorical 

glove_100d = open('/kaggle/input//glove-global-vectors-for-word-representation/glove.6B.100d.txt','r',encoding='utf-8') 
embeddings = {} 
for line in glove_100d : 
    vector = line.split(" ") 
    word = vector[0] 
    coefs = np.asarray([float(val) for val in vector[1:]])
    embeddings[word] = coefs 
glove_100d.close()

In [ ]:
# travailler avec Elmo à la place 
!conda remove -y greenlet
!pip install allennlp

# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
on calcule la similarité cosinus entre premise et hypothesis et on montre que pour neutre, c'est faible 

import math
#source: https://webdevdesigner.com/q/calculate-cosine-similarity-given-2-sentence-strings-67720/
def get_cosine(vec1, vec2):
     intersection = set(vec1) & set(vec2)
     numerator = sum([vec1.count(x) * vec2.count(x) for x in intersection])

     sum1 = sum([vec1.count(x)**2 for x in vec1])
     sum2 = sum([vec2.count(x)**2 for x in vec2])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

similarity_cos = []
for x in copy_train.itertuples():
    encode_prem = encode_sentence(x[2])
    encode_hyp = encode_sentence(x[3])
    similarity_cos.append(get_cosine(encode_prem, encode_hyp))
copy_train["similarité"] = similarity_cos

for i in range(3): 
    plt.subplot(3,2,1 +2*i)
    plt.hist(copy_train[copy_train['label'] == i]["similarité"], bins = 20)
    plt.title(dict_label[i])

print(len(similarity_cos))


### Comparaison des embedidngs des mots (phrases) selon le label 

with strategy.scope():
    # A simple LSTM with glove embeddings and one dense layer
    model1 = Sequential()
    model1.add(Embedding(len(word_index) + 1,
                     100,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

    model1.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
    model1.add(Dense(3, activation='softmax'))
    model1.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model1.summary()

label = to_categorical(copy_train["label"],3)
model1.fit(train_pad, label, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

predictions = [np.argmax(i) for i in model1.predict(test_pad)]

submission = test.id.copy().to_frame()
submission['prediction'] = predictions 
submission.to_csv("submission.csv", index = False)

# GRU with glove embeddings and two dense layers
model2 = Sequential()
model2.add(Embedding(len(word_index) + 1,
             100,
             weights=[embedding_matrix],
             input_length=max_len,
             trainable=False))
model2.add(SpatialDropout1D(0.3))
model2.add(GRU(300))
model2.add(Dense(3, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])   
model2.summary()

label = to_categorical(copy_train["label"],3)
model2.fit(train_pad, label, epochs=5, batch_size=64*strategy.num_replicas_in_sync)
predictions = [np.argmax(i) for i in model2.predict(test_pad)]
submission = test.id.copy().to_frame()
submission['prediction'] = predictions 
submission.to_csv("submission.csv", index = False)

# On entraine maintenant le modele que sur les classes 1 et 2 ( de facon binaire)
sub_copy = copy_train[copy_train["label_bin"]==0]
idx_to_change = [i for i in range(len(predictions)) if predictions[i] == 0]

token.fit_on_texts(list(sub_copy["concatenated"]))
train_sub_seq = token.texts_to_sequences(list(sub_copy["concatenated"]))
#zero pad the sequences : toutes les phrases deviennet de meme longueur(fit avec 0 )
train_sub_pad = sequence.pad_sequences(train_sub_seq, maxlen=max_len)
word_index = token.word_index 
model = Sequential()
model.add(Embedding(len(word_index) + 1, 300, input_length=max_len))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
model.summary()
sub_copy["label"] = sub_copy["label"].apply(lambda x : max(0,x-1))
model.fit(train_sub_pad, sub_copy["label"], epochs=5, batch_size=64*strategy.num_replicas_in_sync) #Multiplying by Strategy to run on TPU's

predict2 =[round(float(i)) for i in model.predict(test_pad[idx_to_change])]

predict2 = [2*i for i in predict2] #[contradictoire == 2]

predictions = pd.Series(predictions)
predictions[idx_to_change] =predict2